<a href="https://colab.research.google.com/github/Nadavo6/BioInformaticsProject/blob/main/mini_project_part_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Proccessing the Database - Part 1
(Habitat, Taxa and COG files)

> Indented block



In [ ]:
# Run this first to Create Data Dictionaries (Habitat, Taxa, Cog_info)

# Prelimanary Code
!pip install biopython
from Bio import SeqIO
from datetime import datetime

# Creating Taxa Dictionary
def process_taxa_file():
    data_dict = {}
    with open(taxa_file_name, 'r') as file:
        for line in file:
            parts = line.strip().split(',')
            strain_id = parts[-3]
            bacteria_id = parts[-4]
            phylum_name = parts[1]
            list = [strain_id, phylum_name]
            if (strain_id!='strain_id'):
              data_dict[bacteria_id] = list
    return data_dict

# # Creating cog_info Dictionary
def process_cog_info_file(file_name, encoding='utf-8'):
    data_dict = {}
    with open(file_name, 'r', encoding=encoding, errors='replace') as file:
        for line_number, line in enumerate(file, start=1):
            try:
                parts = line.strip().split(';')
                if len(parts) == 6:
                    cog = parts[0]
                    key = cog[3:]
                    value = parts[1]+';'+parts[2]+';'+parts[3]+';'+parts[4]
                    data_dict[key] = value

            except UnicodeDecodeError as e:
                print(f"Error decoding line {line_number}: {e}")
                print(f"Problematic line content: {line}")
    return data_dict



# Creating Habitat Dictionary
def process_habitat_file():
    data_dict = {}
    with open(habitat_file_name, 'r') as file:
        for line in file:
            parts = line.strip().split(';')
            host_name = parts[1]
            habitat = parts[-1]
            if (host_name!='bacteria'):
              data_dict[host_name] = habitat
    return data_dict


# Main Code

taxa_file_name = 'taxa.txt'
taxa_dict = process_taxa_file()

habitat_file_name = 'habitat.txt'
habitat_dict = process_habitat_file()

cog_file_name = 'COG_INFO_TABLE.txt'
cog_dict = process_cog_info_file(cog_file_name)

# Extracring Plasmid information - Part 1

In [ ]:
class Genome:
    def __init__(self, Genome_id, creature_name):
        self.Genome_id = Genome_id
        self.creature_name = creature_name
        self.segments = []

def process_file(filename):
    Genomes = {}
    with open(filename, 'r') as file:
        for line in file:
            parts = line.strip().split('#')
            creature_name = parts[3]
            Genome_id = parts[1]
            if Genome_id not in Genomes:
                Genomes[Genome_id] = Genome(Genome_id, creature_name)
            genome = Genomes[Genome_id]
            segments = parts[-1].split()
            current_segment = []
            for segment in segments:
                if segment.isdigit() and len(segment) == 4:
                    current_segment.append(segment)
                else:
                    if current_segment:
                        genome.segments.append(current_segment)
                    current_segment = []
            if current_segment:
                genome.segments.append(current_segment)
    return Genomes

def get_clusters_of_length_d_at_least_q_times(Genomes, d, q):
    segment_counts = {}

    for Genome_id, Genome in Genomes.items():
        for segment in Genome.segments:
            if len(segment) >= d:
                for i in range(len(segment) - d + 1):
                    sub_segment = segment[i:i+d]
                    segment_tuple = tuple(sorted(sub_segment))
                    if segment_tuple in segment_counts:
                        if Genome_id not in segment_counts[segment_tuple]:
                            segment_counts[segment_tuple][Genome_id] = (sub_segment, Genome.creature_name)
                    else:
                        segment_counts[segment_tuple] = {Genome_id: (sub_segment, Genome.creature_name)}

    filtered_clusters = {}
    for segment, genomes in segment_counts.items():
        if len(genomes) >= q:
            filtered_clusters[segment] = genomes
    top_clusters = dict(sorted(filtered_clusters.items(), key=lambda item: len(item[1]), reverse=True)[:5])
    return top_clusters

# Main code
q = 4
d = 2
while (d<5):
  filename = 'cog_words_plasmid.txt'
  start_time = datetime.now()
  Genomes = process_file(filename)

  selected_lists = get_clusters_of_length_d_at_least_q_times(Genomes, d, q)
  timedelta = datetime.now() - start_time
  print(timedelta)
  print()
  print(" "*30 + "Top 20 clusters of length", d, "appearing at least", q, "times:")
  print()
  for segment, cluster_info in selected_lists.items():
      print("Cluster:", segment)
      print("Occurrences:", len(cluster_info))
      print("Genome ID's and Creature Names:")
      for genome_id, (order, creature_name) in cluster_info.items():
          print(f"\tGenome ID: {genome_id}, Creature Name: {creature_name}, Order: {order}")
      print("First Instance of this Cluster:", next(iter(cluster_info.values()))[0])
      print("Information about the COG's:")
      for cog in segment:
        if cog in cog_dict:
          print("\t COG",cog, ":", cog_dict[cog])
        else:
          print("\t COG",cog, ":", "No Information was found for this COG")
      print("Statistics about the different orders we found:")
      word_counter = {}
      host_list = {}
      for value in cluster_info.values():
        word_string = ', '.join(value[0])
        word_counter[word_string] = word_counter.get(word_string, 0) + 1
        host_name = value[1]
        if word_string in host_list:
          host_list[word_string].append(host_name)
        else:
          host_list[word_string]= []
          host_list[word_string].append(host_name)
      for key, value in word_counter.items():
        print("\t [", key , "]:", value, "times out of", len(cluster_info), "which is", round(100*int(value)/len(cluster_info),2), "percent")
      print("Statistics about the Phylum:")
      for order in host_list.keys():
        print("\t Order: [", order , "] - Appears in these Phylum:")
        phylum_counter = {}
        total_hosts = len(host_list[order])
        for host in host_list[order]:
          curr_phylum = taxa_dict[host][1]
          phylum_counter[curr_phylum] = phylum_counter.get(curr_phylum, 0) + 1
        for key, value in phylum_counter.items():
          print("\t\t", key, "-", value, "times out of", total_hosts, ",which is", round(100*int(value)/total_hosts,2), "percent")
      print("Statistics about the Habitat:")
      for order in host_list.keys():
        print("\t Order: [", order , "] - Appears in these Habitats:")
        habitat_counter = {}
        total_hosts = len(host_list[order])
        for host in host_list[order]:
          curr_habitat = habitat_dict[host]
          habitat_counter[curr_habitat] = habitat_counter.get(curr_habitat, 0) + 1
        for key, value in habitat_counter.items():
          print("\t\t", key, "-", value, "times out of", total_hosts, ",which is", round(100*int(value)/total_hosts,2), "percent")
      print()
      print("*"*100)
  d = d+1
  print("*"*100)
  print("*"*100)

In [ ]:
  for segment, cluster_info in selected_lists.items():
      print("Cluster:", segment)
      print("Occurrences:", len(cluster_info))
      print("Genome ID's and Creature Names:")
      for genome_id, (order, creature_name) in cluster_info.items():
          print(f"\tGenome ID: {genome_id}, Creature Name: {creature_name}, Order: {order}")
      print("First Instance of this Cluster:", next(iter(cluster_info.values()))[0])
      print("Information about the COG's:")
      for cog in segment:
        if cog in cog_dict:
          print("\t COG",cog, ":", cog_dict[cog])
        else:
          print("\t COG",cog, ":", "No Information was found for this COG")
      print("Statistics about the different orders we found:")
      word_counter = {}
      host_list = {}
      for value in cluster_info.values():
        word_string = ', '.join(value[0])
        word_counter[word_string] = word_counter.get(word_string, 0) + 1
        host_name = value[1]
        if word_string in host_list:
          host_list[word_string].append(host_name)
        else:
          host_list[word_string]= []
          host_list[word_string].append(host_name)
      for key, value in word_counter.items():
        print("\t [", key , "]:", value, "times out of", len(cluster_info), "which is", round(100*int(value)/len(cluster_info),2), "percent")
      print("Statistics about the Phylum:")
      for order in host_list.keys():
        print("\t Order: [", order , "] - Appears in these Phylum:")
        phylum_counter = {}
        total_hosts = len(host_list[order])
        for host in host_list[order]:
          curr_phylum = taxa_dict[host][1]
          phylum_counter[curr_phylum] = phylum_counter.get(curr_phylum, 0) + 1
        for key, value in phylum_counter.items():
          print("\t\t", key, "-", value, "times out of", total_hosts, ",which is", round(100*int(value)/total_hosts,2), "percent")
      print("Statistics about the Habitat:")
      for order in host_list.keys():
        print("\t Order: [", order , "] - Appears in these Habitats:")
        habitat_counter = {}
        total_hosts = len(host_list[order])
        for host in host_list[order]:
          curr_habitat = habitat_dict[host]
          habitat_counter[curr_habitat] = habitat_counter.get(curr_habitat, 0) + 1
        for key, value in habitat_counter.items():
          print("\t\t", key, "-", value, "times out of", total_hosts, ",which is", round(100*int(value)/total_hosts,2), "percent")
      print()
      print("*"*100)
  d = d+1
  print("*"*100)
  print("*"*100)

# Filtering the Bac file - Part 2
  (Here we clean the chromosomes we don't need from the bac file)

> Indented block

In [ ]:
# Creating host_plasmid List
# This is going over the plasmids, and making a list of every host for their's
def process_plasmid_file(plasmid_file):
    host_list = []
    with open(plasmid_file, 'r') as file:
        for line in file:
          parts = line.strip().split('#')
          host_name = parts[3]
          if host_name not in host_list:
            host_list.append(host_name)
    return host_list

plasmid_file_name = 'cog_words_plasmid.txt'
plasmid_host_list = process_plasmid_file(plasmid_file_name)

In [ ]:
# This Function returns the filtered bac file
def copy_lines_with_condition(input, output, plasmid_search):
    try:
        # Open the bac file in read mode
        with open(input, 'r') as input_file:
            # Filter lines by plasmid_search function
            bacs_that_have_plasmid = [line for line in input_file if plasmid_search(line)]
        # Open a new bac file in write mode
        with open(output_file_name, 'w') as output_file:
            # Write the filtered lines to the new file
            output_file.writelines(bacs_that_have_plasmid)
        return output_file_name

    except FileNotFoundError:
        print(f"The input file '{input_name}' does not exist.")
        return None

# This Function recieves a line, extracts the host's name, and searches for it in plasmid_host list
# returns true if Bac has plasmid, and False if not
def plasmid_search(line):
  chromo_has_plasmid = False
  parts = line.strip().split('#')
  host_name = parts[3]
  for plasmid_host in plasmid_host_list:
    if (plasmid_host == host_name):       ##This checks if this Chromosome's Host has at least one Plasmid
      chromo_has_plasmid = True
  if chromo_has_plasmid:
    return True                   ## True = this host has already a plasmid in our DB
  else:
    return False                ## False = this host does not have a plasmid in our DB


# The main Function
input_file_name = "cog_words_bac.txt"
output_file_name = "new_cog_words_bac.txt"
result_file = copy_lines_with_condition(input_file_name, output_file_name, plasmid_search)

# Downloading the filtered file - will appear in "Downloads"
from google.colab import files
files.download('new_cog_words_bac.txt')


### **Here you have to upload the new_cogs_words_bac file from your Downloads (21.1MB)!**

# Extracring Chromosome information - Part 2

In [ ]:
# REMEMBER TO UPLOAD NEW_COG_WODS_BAC FILE!

class Genome:
    def __init__(self, Genome_id, creature_name):
        self.Genome_id = Genome_id
        self.creature_name = creature_name
        self.segments = []

def process_file(filename):
    Genomes = {}
    with open(filename, 'r') as file:
        for line in file:
            parts = line.strip().split('#')
            creature_name = parts[3]
            Genome_id = parts[1]
            if Genome_id not in Genomes:
                Genomes[Genome_id] = Genome(Genome_id, creature_name)
            genome = Genomes[Genome_id]
            segments = parts[-1].split()
            current_segment = []
            for segment in segments:
                if segment.isdigit() and len(segment) == 4:
                    current_segment.append(segment)
                else:
                    if current_segment:
                        genome.segments.append(current_segment)
                    current_segment = []
            if current_segment:
                genome.segments.append(current_segment)
    return Genomes

def get_clusters_of_length_d_at_least_q_times(Genomes, d, q):
    segment_counts = {}

    for Genome_id, Genome in Genomes.items():
        for segment in Genome.segments:
            if len(segment) >= d:
                for i in range(len(segment) - d + 1):
                    sub_segment = segment[i:i+d]
                    segment_tuple = tuple(sorted(sub_segment))
                    if segment_tuple in segment_counts:
                        if Genome_id not in segment_counts[segment_tuple]:
                            segment_counts[segment_tuple][Genome_id] = (sub_segment, Genome.creature_name)
                    else:
                        segment_counts[segment_tuple] = {Genome_id: (sub_segment, Genome.creature_name)}

    filtered_clusters = {}
    for segment, genomes in segment_counts.items():
        if len(genomes) >= q:
            filtered_clusters[segment] = genomes
    top_clusters = dict(sorted(filtered_clusters.items(), key=lambda item: len(item[1]), reverse=True)[:5])
    return top_clusters

# Main code
q = 5
d = 12
while (d<14):
  filename = 'new_cog_words_bac.txt'
  start_time = datetime.now()
  Genomes = process_file(filename)

  selected_lists = get_clusters_of_length_d_at_least_q_times(Genomes, d, q)
  timedelta = datetime.now() - start_time
  print(timedelta)
  print()
  print(" "*30 + "Top 5 clusters of length", d, "appearing at least", q, "times:")
  print()
  for segment, cluster_info in selected_lists.items():
      print("Cluster:", segment)
      print("Occurrences:", len(cluster_info))
      print("Genome ID's and Creature Names:")
      for genome_id, (order, creature_name) in cluster_info.items():
          print(f"\tGenome ID: {genome_id}, Creature Name: {creature_name}, Order: {order}")
      print("First Instance of this Cluster:", next(iter(cluster_info.values()))[0])
      print("Information about the COG's:")
      for cog in segment:
        if cog in cog_dict:
          print("\t COG",cog, ":", cog_dict[cog])
        else:
          print("\t COG",cog, ":", "No Information was found for this COG")
      print("Statistics about the different orders we found:")
      word_counter = {}
      host_list = {}
      for value in cluster_info.values():
        word_string = ', '.join(value[0])
        word_counter[word_string] = word_counter.get(word_string, 0) + 1
        host_name = value[1]
        if word_string in host_list:
          host_list[word_string].append(host_name)
        else:
          host_list[word_string]= []
          host_list[word_string].append(host_name)
      for key, value in word_counter.items():
        print("\t [", key , "]:", value, "times out of", len(cluster_info), "which is", round(100*int(value)/len(cluster_info),2), "percent")
      print("Statistics about the Phylum:")
      for order in host_list.keys():
        print("\t Order: [", order , "] - Appears in these Phylum:")
        phylum_counter = {}
        total_hosts = len(host_list[order])
        for host in host_list[order]:
          curr_phylum = taxa_dict[host][1]
          phylum_counter[curr_phylum] = phylum_counter.get(curr_phylum, 0) + 1
        for key, value in phylum_counter.items():
          print("\t\t", key, "-", value, "times out of", total_hosts, ",which is", round(100*int(value)/total_hosts,2), "percent")
      print("Statistics about the Habitat:")
      for order in host_list.keys():
        print("\t Order: [", order , "] - Appears in these Habitats:")
        habitat_counter = {}
        total_hosts = len(host_list[order])
        for host in host_list[order]:
          curr_habitat = habitat_dict[host]
          habitat_counter[curr_habitat] = habitat_counter.get(curr_habitat, 0) + 1
        for key, value in habitat_counter.items():
          print("\t\t", key, "-", value, "times out of", total_hosts, ",which is", round(100*int(value)/total_hosts,2), "percent")
      print()
      print("*"*100)
  d = d+1
  print("*"*100)
  print("*"*100)

# Comparing two Profiles of the same Cluster - Part 2

> Indented block

In [ ]:
#functions for profile comaring
def occurence_score(cluster,db1_clusters,db2_clusters,db1_lines,db2_lines):
  score_chrom = len(db1_clusters[cluster])/(db1_lines)
  score_plas = len(db2_clusters[cluster])/(db2_lines)
  return 100-100*abs(score_chrom-score_plas)

def top_three_score(set1,set2):
  common_elements = set1.intersection(set2)
  return ((len(common_elements)*2)/(len(set1)+len(set2)))*100

def phylum_diversity_score(phl_count1,phl_count2):
  return (1-(abs(phl_count1-phl_count2)/max(phl_count1,phl_count2)))*100



def get_top_three_elements(db,db_clusters,cluster, indicator):
    element_count = {};
    for genome_id in db_clusters[cluster].keys():
      curr_element = str(data_by_indicator(db,genome_id,cluster,db_clusters,indicator))
      if curr_element in element_count and curr_element != "-":
        element_count[curr_element] += 1
      else:
        element_count[curr_element] = 1
    return set(sorted(element_count, key=element_count.get, reverse=True)[:3])

def data_by_indicator(db,genome_id,cluster,db_clusters,indicator):
  host_name = db[genome_id].creature_name
  if(indicator == 2): #phylum
    return taxa_dict[host_name][1]
  if(indicator == 4): #hosts
    return host_name
  if(indicator == 5): #habitats
    return habitat_dict[host_name]
  if(indicator == 6): #orders
    return db_clusters[cluster][genome_id][0]

def score(cluster,db1,db2,db1_clusters,db2_clusters,db1_file,db2_file,score_normalize):
  scores = [];
  #get occurence score.
  scores.append(occurence_score(cluster,db1_clusters,db2_clusters,db1_file,db2_file))
  #get score: phylum. indicator 2 (as numbered in the description)
  top3_phylum_db1 = get_top_three_elements(db1,db1_clusters,cluster,2)
  top3_phylum_db2 = get_top_three_elements(db2,db2_clusters, cluster,2)
  scores.append(top_three_score(top3_phylum_db1,top3_phylum_db2))
  #get score: phylum diversity
  phylum_list_db1 = [];
  phylum_list_db2 = [];
  for genome_id in db1_clusters[cluster].keys():
    host_name = db1[genome_id].creature_name
    curr_phylum = taxa_dict[host_name][1]
    if curr_phylum not in phylum_list_db1:
      phylum_list_db1.append(curr_phylum)
    if curr_phylum not in phylum_list_db2:
      phylum_list_db2.append(curr_phylum)
  scores.append(phylum_diversity_score(len(phylum_list_db1),len(phylum_list_db2)))
  #get score:hosts. indicator 4 (as numbered in the description)
  top3_host_names_db1 = get_top_three_elements(db1,db1_clusters,cluster,4)
  top3_host_names_db2 = get_top_three_elements(db2,db2_clusters, cluster,4)
  scores.append(top_three_score(top3_host_names_db1,top3_host_names_db2))
  #get score: habitats. indicator 5 (as numbered in the description)
  top3_habitats_db1 = get_top_three_elements(db1,db1_clusters,cluster,5)
  top3_habitats_db2 = get_top_three_elements(db2,db2_clusters, cluster,5)
  scores.append(top_three_score(top3_habitats_db1,top3_habitats_db2))
  #get score: orders. indicator 6 (as numbered in the description)
  top3_orders_db1 = get_top_three_elements(db1,db1_clusters,cluster,6)
  top3_orders_db2 = get_top_three_elements(db2,db2_clusters, cluster,6)
  scores.append(top_three_score(top3_habitats_db1,top3_habitats_db2))
  #calculate final score:
  final_score = 0
  for i in range(5):
    final_score += scores[i]*score_normalize[i]
  return final_score


def get_all_clusters_of_length_d_at_least_q_times(Genomes, d, q):
    segment_counts = {}
    for Genome_id, Genome in Genomes.items():
        for segment in Genome.segments:
            if len(segment) >= d:
                for i in range(len(segment) - d + 1):
                    sub_segment = segment[i:i+d]
                    segment_tuple = tuple(sorted(sub_segment))
                    if segment_tuple in segment_counts:
                        if Genome_id not in segment_counts[segment_tuple]:
                            segment_counts[segment_tuple][Genome_id] = (sub_segment, Genome.creature_name)
                    else:
                        segment_counts[segment_tuple] = {Genome_id: (sub_segment, Genome.creature_name)}

    filtered_clusters = {}
    for segment, genomes in segment_counts.items():
        if len(genomes) >= q:
            filtered_clusters[segment] = genomes
    return filtered_clusters



In [ ]:
#data printing function:
def print_data(db,cluster,cluster_dict):
  print("Numbers of appearances: ")
  print(len(cluster_dict[cluster].keys()))
  print("Top 3 phylums:")
  print(get_top_three_elements(db,cluster_dict,cluster,2))
  phylum_list = [];
  for genome_id in cluster_dict[cluster].keys():
    host_name = db[genome_id].creature_name
    curr_phylum = taxa_dict[host_name][1]
    if curr_phylum not in phylum_list:
      phylum_list.append(curr_phylum)
  print("number of phylums: ")
  print(len(phylum_list))
  print("\n")
  print("Top 3 hosts: ")
  print(get_top_three_elements(db,cluster_dict,cluster,4))
  print("\n")
  print("Top 3 habitat: ")
  print( get_top_three_elements(db,cluster_dict,cluster,5))
  print("\n")
  print("Top 3 orders: ")
  print(get_top_three_elements(db,cluster_dict,cluster,6))

In [ ]:
def get_clusters_that_differ_significantly(db1,db2,q1,q2,d,db1_lines,db2_lines,score_normalize,bar_score):
  db1_clusters = get_all_clusters_of_length_d_at_least_q_times(db1,d,q1)
  db2_clusters = get_all_clusters_of_length_d_at_least_q_times(db2,d,q2)
  #get the clusters found in both maps:
  common_clusters = set(db1_clusters.keys()) & set(db2_clusters.keys())
  for cluster in common_clusters:
    curr_score = score(cluster,db1,db2,db1_clusters,db2_clusters,db1_lines,db2_lines,score_normalize)

    if curr_score<bar_score:
      return [cluster,db1_clusters,db2_clusters]

  #if no fitting cluster was found. we return 0
  return 0


#main code:
q1 = 316
q2 = 54
d = 4
score_normalize = [0.1,0.2,0.1,0.2,0.2,0.2]
bar_score = 40
chromosome_filename = 'new_cog_words_bac.txt'
plasmid_filename = 'cog_words_plasmid.txt'
chromosome_file = open(chromosome_filename, 'r')
plasmid_file = open(plasmid_filename, 'r')
lines_chromosome =sum(1 for line in chromosome_file)
lines_plasmid =sum(1 for line in plasmid_file)
chromosome_genomes = process_file(chromosome_filename)
plasmid_genomes = process_file(plasmid_filename)
cluster  = get_clusters_that_differ_significantly(chromosome_genomes,plasmid_genomes,q1,q2,d,lines_chromosome,lines_plasmid,score_normalize,bar_score)
if(cluster == 0):
  print("No cluster found! change query")
else:
  print(" "*30 + "Cluster found: ")
  print(" "*28 , cluster[0])
  print("Data from " + chromosome_filename + ": ")
  print_data(chromosome_genomes,cluster[0],cluster[1])
  print("\n")
  print("\n")
  print("Data from " + plasmid_filename + ": ")
  print_data(plasmid_genomes,cluster[0],cluster[2])